In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
# -----------------------
# 1. Config
# -----------------------

CSV_PATH = "/content/sample_data/merged.csv"  # <- change if needed
RANDOM_SEED = 42
BATCH_SIZE = 32
MAX_LEN = 64            # you can tune this
EMBED_DIM = 128
N_HEAD = 4
NUM_LAYERS = 2
FF_DIM = 256
DROPOUT = 0.1
NUM_EPOCHS = 10
LR = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

Using device: cpu


In [ ]:
# -----------------------
# 2. Load data
# -----------------------

df = pd.read_csv(CSV_PATH)
df['label'] = df['label'].str.upper()
df = df[df['label'].str.lower() != 'label']
df = df[df['label'].str.lower() != 'api_failure']
# expecting columns: 'comment', 'label'
print(df.head())


                                                text        label
0                                      НАЦИСТ ФАШИСТ    OFFENSIVE
1  Хачики попутали че то. Надо им Уржар 2.0 устроить  HATE_SPEECH
2                                            Мал шщс    OFFENSIVE
3  Не тащите эту хуету сюда, пожалуйста, уже в тр...    OFFENSIVE
4                              Барбагыла акмактарга!    OFFENSIVE


In [ ]:
# -----------------------
# 3. EDA (label distribution, imbalance)
# -----------------------

label_counts = df["label"].value_counts()
print("\nLabel counts:")
print(label_counts)

print("\nLabel proportions:")
print((label_counts / len(df)).round(3))

# Imbalance ratio (max_count / min_count)
imbalance_ratio = label_counts.max() / label_counts.min()
print(f"\nImbalance ratio (max/min): {imbalance_ratio:.2f}")

# Comment length statistics
df["text_len"] = df["text"].astype(str).apply(lambda x: len(x.split()))
print("\nText length stats (in tokens):")
print(df["text_len"].describe())


Label counts:
label
NORMAL         1052
HATE_SPEECH     498
OFFENSIVE       396
Name: count, dtype: int64

Label proportions:
label
NORMAL         0.541
HATE_SPEECH    0.256
OFFENSIVE      0.203
Name: count, dtype: float64

Imbalance ratio (max/min): 2.66

Text length stats (in tokens):
count    1946.000000
mean        9.843782
std        11.683234
min         1.000000
25%         4.000000
50%         7.000000
75%        11.000000
max       201.000000
Name: text_len, dtype: float64


In [ ]:
# -----------------------
# 4. Encode labels
# -----------------------

label2id = {label: idx for idx, label in enumerate(sorted(df["label"].unique()))}
id2label = {v: k for k, v in label2id.items()}
print("\nLabel mapping:", label2id)

df["label_id"] = df["label"].map(label2id)


Label mapping: {'HATE_SPEECH': 0, 'NORMAL': 1, 'OFFENSIVE': 2}


In [ ]:
# -----------------------
# 5. Train/val split
# -----------------------
train_df, val_df = train_test_split(
    df,
    test_size = 0.2,
    random_state=RANDOM_SEED,
    stratify=df["label_id"],
    )

print("\nTrain size:", len(train_df), "Val size:", len(val_df))


Train size: 1556 Val size: 390


In [ ]:
# -----------------------
# 6. Tokenization & Vocabulary
# -----------------------

def basic_tokenize(text: str):
    # Simple whitespace tokenizer, lowercasing
    # You can improve: remove extra spaces, keep punctuation as tokens, etc.
    text = str(text).lower().strip()
    # optional: replace multiple spaces
    text = re.sub(r"\s+", " ", text)
    return text.split()

# Build vocabulary on train set only
special_tokens = ["<pad>", "<unk>", "<cls>"]
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"
CLS_TOKEN = "<cls>"

# Collect tokens
all_tokens = []
for text in train_df["text"]:
    toks = basic_tokenize(text)
    all_tokens.extend(toks)

# Frequency threshold if you want to drop rare words
token_freq = Counter(all_tokens)
min_freq = 2  # you can tune this (or set to 1 to keep everything)

vocab_tokens = [tok for tok, c in token_freq.items() if c >= min_freq]
vocab = special_tokens + sorted(vocab_tokens)

token2id = {tok: idx for idx, tok in enumerate(vocab)}
id2token = {idx: tok for tok, idx in token2id.items()}

PAD_IDX = token2id[PAD_TOKEN]
UNK_IDX = token2id[UNK_TOKEN]
CLS_IDX = token2id[CLS_TOKEN]

print("\nVocab size:", len(vocab))

def encode_text(text: str, max_len: int = MAX_LEN):
    tokens = basic_tokenize(text)
    # prepend CLS token
    tokens = [CLS_TOKEN] + tokens

    # convert to ids
    ids = [token2id.get(tok, UNK_IDX) for tok in tokens]

    # truncate
    ids = ids[:max_len]

    # pad
    attention_mask = [1] * len(ids)
    if len(ids) < max_len:
        pad_len = max_len - len(ids)
        ids = ids + [PAD_IDX] * pad_len
        attention_mask = attention_mask + [0] * pad_len

    return ids, attention_mask


Vocab size: 1814


In [ ]:
# -----------------------
# 7. Dataset & DataLoader
# -----------------------

class HateSpeechDataset(Dataset):
    def __init__(self, df):
        self.texts = df["text"].tolist()
        self.labels = df["label_id"].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        input_ids, attention_mask = encode_text(text, max_len=MAX_LEN)
        input_ids = torch.tensor(input_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        label = torch.tensor(label, dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": label
        }

train_dataset = HateSpeechDataset(train_df)
val_dataset = HateSpeechDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# -----------------------
# 8. Transformer Encoder model
# -----------------------

class TransformerClassifier(nn.Module):
    def __init__(
        self,
        vocab_size,
        num_classes,
        d_model=EMBED_DIM,
        nhead=N_HEAD,
        num_layers=NUM_LAYERS,
        dim_feedforward=FF_DIM,
        dropout=0.1,
        max_len=MAX_LEN,
        pad_idx=PAD_IDX,
    ):
        super().__init__()
        self.d_model = d_model
        self.pad_idx = pad_idx
        self.max_len = max_len

        self.token_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_embedding = nn.Embedding(max_len, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,  # easier: (batch, seq, dim)
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, input_ids, attention_mask=None):
        """
        input_ids: (batch, seq_len)
        attention_mask: (batch, seq_len) with 1 for tokens, 0 for padding
        """
        batch_size, seq_len = input_ids.shape

        # token embeddings
        tok_emb = self.token_embedding(input_ids)  # (batch, seq_len, d_model)

        # positional embeddings
        positions = torch.arange(0, seq_len, device=input_ids.device).unsqueeze(0)
        pos_emb = self.pos_embedding(positions)  # (1, seq_len, d_model)

        x = tok_emb + pos_emb

        # src_key_padding_mask: (batch, seq_len) True where padding
        if attention_mask is not None:
            src_key_padding_mask = attention_mask == 0
        else:
            src_key_padding_mask = (input_ids == self.pad_idx)

        x = self.transformer_encoder(
            x,
            src_key_padding_mask=src_key_padding_mask
        )  # (batch, seq_len, d_model)

        # Use CLS token representation (first token)
        cls_out = x[:, 0, :]  # (batch, d_model)
        cls_out = self.dropout(cls_out)
        logits = self.fc(cls_out)  # (batch, num_classes)
        return logits

num_classes = len(label2id)
model = TransformerClassifier(
    vocab_size=len(vocab),
    num_classes=num_classes,
    d_model=EMBED_DIM,
    nhead=N_HEAD,
    num_layers=NUM_LAYERS,
    dim_feedforward=FF_DIM,
    dropout=DROPOUT,
    max_len=MAX_LEN,
    pad_idx=PAD_IDX,
).to(device)

print(model)

TransformerClassifier(
  (token_embedding): Embedding(1814, 128, padding_idx=0)
  (pos_embedding): Embedding(64, 128)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=3, bias=True)
)


In [ ]:
# -----------------------
# 9. Handle class imbalance (weights)
# -----------------------

train_label_counts = train_df["label_id"].value_counts().sort_index().values
# Inverse frequency
class_weights = 1.0 / train_label_counts
class_weights = class_weights / class_weights.sum() * len(class_weights)  # normalize-ish
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

print("\nClass weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)



Class weights: tensor([1.0994, 0.5203, 1.3803])


In [ ]:
# -----------------------
# 10. Training & Evaluation loops
# -----------------------

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask=attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * input_ids.size(0)

    avg_loss = total_loss / len(dataloader.dataset)
    return avg_loss

def evaluate(model, dataloader, device):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, attention_mask=attention_mask)
            loss = criterion(logits, labels)

            total_loss += loss.item() * input_ids.size(0)

            preds = torch.argmax(logits, dim=-1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    avg_loss = total_loss / len(dataloader.dataset)
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="macro", zero_division=0
    )
    report = classification_report(
        all_labels, all_preds,
        target_names=[id2label[i] for i in range(num_classes)],
        zero_division=0
    )
    return avg_loss, acc, precision, recall, f1, report


In [ ]:
# -----------------------
# 11. Train loop
# -----------------------

best_val_f1 = 0.0
for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, val_prec, val_rec, val_f1, val_report = evaluate(model, val_loader, device)

    print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
    print(f"  Train loss: {train_loss:.4f}")
    print(f"  Val loss  : {val_loss:.4f}")
    print(f"  Val Acc   : {val_acc:.4f}")
    print(f"  Val Prec  : {val_prec:.4f}")
    print(f"  Val Recall: {val_rec:.4f}")
    print(f"  Val F1    : {val_f1:.4f}")

    # Save best model by F1
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_transformer_hatespeech.pt")
        print("  🔥 New best model saved!")

print("\nFinal evaluation on validation set (best epoch metrics above):")
print(val_report)

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:515: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(



Epoch 1/10
  Train loss: 1.0456
  Val loss  : 0.9423
  Val Acc   : 0.5949
  Val Prec  : 0.7357
  Val Recall: 0.5789
  Val F1    : 0.5477
  🔥 New best model saved!

Epoch 2/10
  Train loss: 0.8058
  Val loss  : 0.7603
  Val Acc   : 0.6846
  Val Prec  : 0.6448
  Val Recall: 0.6527
  Val F1    : 0.6419
  🔥 New best model saved!

Epoch 3/10
  Train loss: 0.6060
  Val loss  : 0.7793
  Val Acc   : 0.6718
  Val Prec  : 0.6466
  Val Recall: 0.6694
  Val F1    : 0.6450
  🔥 New best model saved!

Epoch 4/10
  Train loss: 0.4451
  Val loss  : 0.9257
  Val Acc   : 0.6231
  Val Prec  : 0.6320
  Val Recall: 0.6403
  Val F1    : 0.6046

Epoch 5/10
  Train loss: 0.3429
  Val loss  : 1.0012
  Val Acc   : 0.7282
  Val Prec  : 0.6926
  Val Recall: 0.6674
  Val F1    : 0.6771
  🔥 New best model saved!

Epoch 6/10
  Train loss: 0.2635
  Val loss  : 1.0555
  Val Acc   : 0.7026
  Val Prec  : 0.6635
  Val Recall: 0.6823
  Val F1    : 0.6710

Epoch 7/10
  Train loss: 0.2119
  Val loss  : 1.2234
  Val Acc   : 